In [1]:
import pandas as pd

brazil_bond = pd.read_csv('Brazil-6M.csv')
brazil_bond = brazil_bond[brazil_bond['Date'] >= '2024-10-11']
brazil_bond['Close'] = brazil_bond['Close']/100


In [2]:
# Convert the date format from ISO format with timezone to US date format
brazil_bond['Date'] = pd.to_datetime(brazil_bond['Date']).dt.strftime('%Y-%m-%d')
brazil_bond

,Date,Open,High,Low,Close,% Change,% Change vs Average,Volume
122,2024-10-13,11.70,11.70,11.64,0.1164,-0.43,-0.57,0
123,2024-10-14,11.66,11.66,11.66,0.1166,0.13,-0.02,0
124,2024-10-15,11.72,11.72,11.66,0.1166,0.04,-0.10,0
125,2024-10-16,11.71,11.71,11.71,0.1171,0.39,0.24,0
126,2024-10-17,11.71,11.72,11.71,0.1172,0.09,-0.06,0
...,...,...,...,...,...,...,...,...
238,2025-04-06,14.57,14.59,14.51,0.1457,-0.03,-0.18,0
239,2025-04-07,14.58,14.61,14.51,0.1461,0.21,0.06,0
240,2025-04-08,14.61,14.66,14.58,0.1460,-0.02,-0.17,0
241,2025-04-09,14.64,14.66,14.62,0.1462,0.12,-0.02,0


In [3]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

def calculate_bond_prices(df, face_value, maturity_date):
    """
    df: DataFrame with columns ['Date', 'Close'] where Date is datetime, Close is annual close of interest rate 
    face_value: 액면가
    maturity_date: 만기일 (datetime.date)
    """
    prices = []

    for _, row in df.iterrows():
        current_date = row['Date']
        r = row['Close']
        current_date = datetime.strptime(current_date, '%Y-%m-%d').date()
        t = (maturity_date - current_date).days / 365

        price = face_value / ((1 + r) ** t)
        prices.append(price)

    df['price'] = prices
    return df

# 만기일 설정
maturity = datetime.strptime('2025-04-11', '%Y-%m-%d').date()

# 계산 실행
result_df = calculate_bond_prices(brazil_bond, face_value=1000, maturity_date=maturity)

In [4]:
result_df["Close"] = result_df["price"]
result_df.drop(columns=["price"], inplace=True)

In [5]:
result_df.to_csv('brazil_bond_prices.csv', index=False)

In [6]:
import pandas as pd

# Read the CSV files
brl_usd = pd.read_csv('brl_usd.csv')
krw_usd = pd.read_csv('krw_usd.csv')

# Convert Date columns to datetime
brl_usd['Date'] = pd.to_datetime(brl_usd['Date'])
krw_usd['Date'] = pd.to_datetime(krw_usd['Date'])

# Merge the dataframes on Date
merged = pd.merge(brl_usd, krw_usd, on='Date', suffixes=('_brl', '_krw'))

# Calculate BRL/KRW rate (BRL/USD divided by KRW/USD)
merged['BRL_KRW'] = merged['Close_brl'] / merged['Close_krw']

# Create output dataframe with Date and BRL/KRW rate
brl_krw = merged[['Date', 'BRL_KRW']]

# Save to CSV
brl_krw.to_csv('brl_krw.csv', index=False)

# Print first few rows
print("First few rows of calculated BRL/KRW rates:")
print(brl_krw.head())

First few rows of calculated BRL/KRW rates:
        Date     BRL_KRW
0 2024-10-11  241.815222
1 2024-10-14  240.318318
2 2024-10-15  242.599255
3 2024-10-16  241.185433
4 2024-10-17  240.684527


In [7]:
brl_krw = pd.read_csv("brl_krw.csv", index_col="Date")
brazil_6m = pd.read_csv("brazil_bond_prices.csv", index_col="Date")


all_dates = brazil_6m.index.union(brl_krw.index)

# # 모든 데이터프레임을 합집합 날짜로 재인덱싱하고 이전 값으로 채우기
brazil_6m = brazil_6m.reindex(all_dates)
brazil_6m.ffill(inplace=True)
brazil_6m.bfill(inplace=True)

brazil_6m

,Open,High,Low,Close,% Change,% Change vs Average,Volume
Date,,,,,,,
2024-10-11,11.70,11.70,11.64,947.147504,-0.43,-0.57,0.0
2024-10-13,11.70,11.70,11.64,947.147504,-0.43,-0.57,0.0
2024-10-14,11.66,11.66,11.66,947.350046,0.13,-0.02,0.0
2024-10-15,11.72,11.72,11.66,947.636340,0.04,-0.10,0.0
2024-10-16,11.71,11.71,11.71,947.716952,0.39,0.24,0.0
...,...,...,...,...,...,...,...
2025-04-07,14.58,14.61,14.51,998.506706,0.21,0.06,0.0
2025-04-08,14.61,14.66,14.58,998.880537,-0.02,-0.17,0.0
2025-04-09,14.64,14.66,14.62,999.252597,0.12,-0.02,0.0


In [8]:
# # 모든 데이터프레임을 합집합 날짜로 재인덱싱하고 이전 값으로 채우기
brl_krw = brl_krw.reindex(all_dates)
brl_krw.ffill(inplace=True)
brl_krw.bfill(inplace=True)

brl_krw


,BRL_KRW
Date,
2024-10-11,241.815222
2024-10-13,241.815222
2024-10-14,240.318318
2024-10-15,242.599255
2024-10-16,241.185433
...,...
2025-04-07,249.745287
2025-04-08,248.571083
2025-04-09,247.140508


In [9]:
BRL_USD_CONTRACT_SIZE = 100000 # 단위 BRL

brazil_6m_krw = pd.DataFrame(brazil_6m["Close"] * brl_krw["BRL_KRW"], index=all_dates, columns=["Close"])

brazil_6m_krw.to_csv("brazil_bond_prices_by_krw.csv")





In [10]:
brl_krw = pd.read_csv("brl_krw.csv", index_col="Date")
brl_usd = pd.read_csv("brl_usd.csv", index_col="Date")


all_dates = brl_usd.index.union(brl_krw.index)

# # 모든 데이터프레임을 합집합 날짜로 재인덱싱하고 이전 값으로 채우기
brl_usd = brl_usd.reindex(all_dates)
brl_usd.ffill(inplace=True)
brl_usd.bfill(inplace=True)

brl_usd

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-10-11,0.179157,0.176938,0.179892,0.179157,inf,inf,inf
2024-10-14,0.178234,0.177010,0.179224,0.178234,inf,inf,inf
2024-10-15,0.178741,0.176888,0.179173,0.178741,inf,inf,inf
2024-10-16,0.176929,0.175605,0.177650,0.176929,inf,inf,inf
2024-10-17,0.176429,0.175889,0.176813,0.176429,inf,inf,inf
...,...,...,...,...,...,...,...
2025-04-07,0.171232,0.168762,0.172432,0.171232,inf,inf,inf
2025-04-08,0.169102,0.166872,0.170605,0.169102,inf,inf,inf
2025-04-09,0.166298,0.163447,0.166598,0.166298,inf,inf,inf


In [11]:
brl_krw = brl_krw.reindex(all_dates)
brl_krw.ffill(inplace=True)
brl_krw.bfill(inplace=True)

brl_krw

,BRL_KRW
Date,
2024-10-11,241.815222
2024-10-14,240.318318
2024-10-15,242.599255
2024-10-16,241.185433
2024-10-17,240.684527
...,...
2025-04-07,249.745287
2025-04-08,248.571083
2025-04-09,247.140508


In [12]:
BRL_USD_CONTRACT_SIZE = 100000 # 단위 BRL

brl_usd_by_krw = pd.DataFrame(brl_usd["Close"] * brl_krw["BRL_KRW"] * BRL_USD_CONTRACT_SIZE, index=all_dates, columns=["Close"])

brl_usd_by_krw.to_csv("brl_usd_by_krw_contract_size.csv")





In [13]:
KRW_USD_CONTRACT_SIZE = 125000000 # 단위 KRW

krw_usd = pd.read_csv("krw_usd.csv", index_col="Date")
krw_usd["Close"] = krw_usd["Close"] * KRW_USD_CONTRACT_SIZE

krw_usd.to_csv("krw_usd_by_contract_size.csv")